In [1]:
import re
import pandas as pd
import numpy as np

##Pre-processing

In [3]:
df = pd.read_csv('/content/drive/MyDrive/NLP Assignments/A1_dataset.csv')

In [4]:
#download
import spacy
nlp = spacy.load("en_core_web_sm")
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('averaged_perceptron_tagger')
# !pip install contextualSpellCheck
# import contextualSpellCheck
# nlp_2 = spacy.load('en_core_web_sm')
# contextualSpellCheck.add_to_pipe(nlp_2)
from nltk.tokenize.treebank import TreebankWordDetokenizer
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import wordnet


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [5]:
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None

In [6]:
stop_list=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [50]:
def processing(sentence, verbose=False):
  # Text_org = df.loc[i, "TEXT"]
  p = re.compile(r'<.*?>')
  rmhtml_Text=p.sub('',sentence)  #remove HTML Tags
  # rmurl_Text = re.sub(r'http\S+|https:\S+|www\S+|.com\S+', '', rmhtml_Text) #remove URL tags
  pre_url = r'(?:https?://)|(?:www\.)'
  url_set = r'[\w+=?/._\-:;&*^$#@~`!|]'
  rmurl_Text=re.sub(f'(?:{pre_url}){url_set}+|{url_set}+\.com{url_set}*', '',rmhtml_Text)
  # rmusername_Text= re.sub('@[^\s]+','',rmurl_Text)#remove usernames
  rmwhite_Text=re.sub(' +', ' ',rmurl_Text) #remove extra whitespace
  token_text=[token.text for token in nlp(rmwhite_Text)]#tokenize text 
  rmpunt_text=[word for word in token_text if word.isalnum()] #punctuation and username removal
  token_pos=nltk.pos_tag(rmpunt_text)
  lem_text=[]
  # print(token_pos)
  for word in token_pos:
        pos = pos_tagger(word[1])
        if pos:
          lem_text.append(lemmatizer.lemmatize(word[0], pos)) #lemmatize word based on part-of-speech tagging
        else:
          lem_text.append(lemmatizer.lemmatize(word[0])) #lemmatize word without part-of-speech tagging
  # stem_text= [stemmer.stem(word) for word in rmpunt_text]#stemming
  # lem_text= [lemmatizer.lemmatize(word)for word in rmpunt_text] 
  rmstop_text=[i for i in lem_text if i not in stop_list]#removing stop words
  if len(rmstop_text)==0:
    rmstop_text= ['<s>','</s>']
  if rmstop_text[0] != '<s>' and len(rmstop_text)!=0:
    rmstop_text.insert(0, '<s>')
  if rmstop_text[-1] != '</s>' and len(rmstop_text)!=0:
    rmstop_text.append('</s>')
  
  sentence=TreebankWordDetokenizer().detokenize(rmstop_text) #convert to sentence 
  # print(sentence)
  
  if verbose:
    print(f'Removing HTML tags:{rmhtml_Text}')
    print(f'Removing URL tags:{rmurl_Text}')
    # print(f'Removing username: {rmusername_Text}')
    print(f'Removing whitespace:{rmwhite_Text}')
    print(f'Tokenize Text :{token_text}')
    print(f'Punctuation removal: {rmpunt_text}')
    print(f'Lemmatize Text :{lem_text}')
    print(f'Removing stop words Text :{rmstop_text}')
    # print(f'Spelling correction :{autospell(sentence)}')
    print(token_text)
  return sentence

In [51]:
test="</a> /// http://bit.ly/etD3a /// @Luke Oh Luke  , why don't you answer me? pleease try it . Love your music ! Hear it the whole day, really trying !   "
processing(test, verbose=True)

Removing HTML tags: /// http://bit.ly/etD3a /// @Luke Oh Luke  , why don't you answer me? pleease try it . Love your music ! Hear it the whole day, really trying !   
Removing URL tags: ///  /// @Luke Oh Luke  , why don't you answer me? pleease try it . Love your music ! Hear it the whole day, really trying !   
Removing whitespace: /// /// @Luke Oh Luke , why don't you answer me? pleease try it . Love your music ! Hear it the whole day, really trying ! 
Tokenize Text :[' ', '///', '///', '@Luke', 'Oh', 'Luke', ',', 'why', 'do', "n't", 'you', 'answer', 'me', '?', 'pleease', 'try', 'it', '.', 'Love', 'your', 'music', '!', 'Hear', 'it', 'the', 'whole', 'day', ',', 'really', 'trying', '!']
Punctuation removal: ['Oh', 'Luke', 'why', 'do', 'you', 'answer', 'me', 'pleease', 'try', 'it', 'Love', 'your', 'music', 'Hear', 'it', 'the', 'whole', 'day', 'really', 'trying']
Lemmatize Text :['Oh', 'Luke', 'why', 'do', 'you', 'answer', 'me', 'pleease', 'try', 'it', 'Love', 'your', 'music', 'Hear', 'i

'<s> Oh Luke answer pleease try Love music Hear whole day really try </s>'

In [52]:
data=df.apply(lambda row: processing(row.TEXT,verbose=False),axis=1)

In [ ]:
data

In [54]:
df['NEW_TEXT']=data

In [55]:
df.to_csv('A2_dataset_processed.csv')

##Q1

In [56]:
processed_df =pd.read_csv('/content/drive/MyDrive/NLP Assignments/A2_dataset_processed.csv')

In [ ]:
processed_df['NEW_TEXT']

##Q2